<a href="https://colab.research.google.com/github/CattoYT/ValorantCommsBot/blob/rewrite/yolo_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install websockets
!pip install ultralytics pillow
!pip install pyngrok

#from google.colab import drive
#drive.mount("/content/drive")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadat

In [3]:
import asyncio
from websockets.server import serve
import ultralytics
import numpy as np
from PIL import Image

from pyngrok import conf, ngrok
from google.colab import userdata

conf.get_default().auth_token = userdata.get('NGROKToken')

public_url = ngrok.connect("8765", domain="cheaply-caring-pup.ngrok-free.app").public_url# change this domain

def overlayCensor(self, img):
        img_np = np.array(img)
        height, width, _ = img_np.shape
        box_coords = (width - 70, 93, width - 30, 400)  # (x_min, y_min, x_max, y_max)

        # Set the specified area to black
        img_np[box_coords[1]:box_coords[3], box_coords[0]:box_coords[2]] = [0, 0, 0]
        img = Image.fromarray(img_np)
        return img

async def infer(websocket):
    async for message in websocket:

        img = np.array(message)

        Image.frombytes('RGB', img.size, img.bgra, 'raw', 'BGRX').save("yes.png")


        await websocket.send("ready")


async def main():
    async with serve(infer, "localhost", 8765):
        await asyncio.Future()  # run forever



await main()

OSError: [Errno 99] error while attempting to bind on address ('::1', 1834, 0, 0): cannot assign requested address

In [ ]:
import os

async def yoloV8Detection(self):
      # This needs to run as fast as possible cuz valorant is fast asf
      import logging
      # Suppress logging output
      if self.model == None:
          modelPath = "Models/epoch270.pt"
          self.model = ultralytics.YOLO(modelPath)
          # converts the model to engine if it isnt already
          print(modelPath[:-3] + ".engine")
          if modelPath[-2:] == "pt":
              try:
                  self.model = ultralytics.YOLO(modelPath[:-3] + ".engine")
              except:
                  if input("Do you want to convert this model to "
                           "TensorRT for faster inference? (y/n): ").lower() == "y":
                      self.model.export(dynamic=True, format="engine")
                      os.remove(modelPath[:-3] + ".onnx")
                      self.model = ultralytics.YOLO(modelPath[:-3] + ".engine")
      cooldown = 0
      logging.getLogger('ultralytics').setLevel(logging.WARNING) #chatgpt'd because the documentation is kinda shit
          screenshot = detectors.capture_screenshot()
          results = self.model(self.overlayCensor(screenshot), conf=0.70, device="0")
          detections = results[0].boxes
          class_ids = detections.cls.cpu().numpy() if detections.cls is not None else [] # thanks copilot
          detected = 0
          for class_id in class_ids:
              if class_id == 1.0: # check if the class id is actually an enemy
                  detected += 1
          if detected >= 1:
              if time.time() - cooldown > 30:  # 30 seconds
                  cooldown = time.time()
                  spk.sayVoice("voices/mio/new-round/respawn - more_enemies_what_do_we_do.wav")